In [1]:
import sys, os
import torch, wandb
import torch.nn as nn
from torch.utils.data import DataLoader
sys.path.append(os.path.abspath(os.path.join(os.curdir, '..')))

In [2]:
from configs import example_unet_config as config
from models.example_unet import SimpleUNet
from datasets.depth_dataset import DepthDataset
from utils.train_utils import train_model

In [3]:
# Set a fixed random seed for reproducibility
torch.manual_seed(config.random_seed)

train_full_dataset = DepthDataset(
    data_dir=os.path.join(config.dataset_path, 'train/train'),
    list_file=os.path.join(config.dataset_path, 'train_list.txt'), 
    transform=config.transform_train,
    target_transform=config.target_transform,
    has_gt=True)
    
    # Create test dataset without ground truth
test_dataset = DepthDataset(
    data_dir=os.path.join(config.dataset_path, 'test/test'),
    list_file=os.path.join(config.dataset_path, 'test_list.txt'),
    transform=config.transform_val,
    has_gt=False)  # Test set has no ground truth
    
# Split training dataset into train and validation
total_size = len(train_full_dataset)
train_size = int((1-config.val_part) * total_size)  
val_size = total_size - train_size    
    
train_dataset, val_dataset = torch.utils.data.random_split(
    train_full_dataset, [train_size, val_size]
)
#val_dataset.transform = config.transform_val # I dont think we need to use augmentations for validation

# Create data loaders with memory optimizations
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.train_bs, 
    shuffle=True, 
    num_workers=config.num_workers, 
    pin_memory=True,
    drop_last=True,
    persistent_workers=True
)
    
    
val_loader = DataLoader(
    val_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)
    
test_loader = DataLoader(
    test_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")

Train size: 20375, Validation size: 3596, Test size: 650


In [4]:
model = config.model()
# #model = nn.DataParallel(model)


optimizer = config.optimizer(model.parameters())
print(f"Using device: {config.device}")

Using device: cuda:3


In [5]:
print("Starting training...")
exp_name = "sanity-check-2"
with wandb.init(project="CIL",
                save_code=True,
                notes=config.WANDB_NOTES):
    model = train_model(model, train_loader, val_loader,
                        config.loss, optimizer, config.epochs, config.device,
                       exp_path=os.path.join(config.dataset_path, exp_name))

Starting training...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: noloo. Use `wandb login --relogin` to force relogin


Epoch 1/1


Validation: 100%|███████████████████████████████████████████████████████████████| 899/899 [01:17<00:00, 11.66it/s]


Train Loss: 2.2114, Validation Loss: 1.9542
New best model saved at epoch 1 with validation loss: 1.9542

Best model was from epoch 1 with validation loss: 1.9542


train/train,▁
val/val,▁
train/train,2.21137
val/val,1.95415


In [6]:
import utils.train_utils as tu

In [7]:
from utils.train_utils import evaluate_model
import importlib
importlib.reload(tu)
tu.evaluate_model(model, val_loader, config.device,
                  exp_path=os.path.join(config.dataset_path, exp_name))

Evaluating: 100%|███████████████████████████████████████████████████████████████| 899/899 [01:30<00:00,  9.93it/s]


{'MAE': 1.0475800055623181,
 'RMSE': 1.3984738329041144,
 'siRMSE': 0.3276368068661686,
 'REL': 0.4536396080077634,
 'Delta1': 0.3889984628758834,
 'Delta2': 0.6706351533841537,
 'Delta3': 0.842293435483125}

SyntaxError: invalid syntax (4182258668.py, line 1)